# Monetization workflow: first full implementation

This notebook is a first try at implementing the full monetization workflow:
1. Create a mapping that allows for subtracting impacts (i.e. that features activities on multiple levels).
2. Take a subset of that mapping, using a simple example structure and using only activities that are mapping to a single ecoinvent activity
3. Regionalize (using premise function?)
4. Calculate impacts, using subtraction approach C (see [approaches](https://pad.degrowth.net/SVQ10BiDR-ed7aX0oeOqiA#))
6. Get activity size from `fulldata.gdx`
7. Apply monetization factors
8. Money $$$

Note: There are a lot of details (e.g. hard coal / lignite mix) to be taken care of, and a lot of things to be refined (e.g. mappings) later.

## 1. Create mapping between REMIND and ecoinvent

Already present in an Excel sheet. Load it:

In [1]:
import pandas as pd

In [2]:
pe2se = pd.read_excel(
    "../mappings/mapping.xlsx", sheet_name="pe2se", nrows=48, 
    usecols=["abbreviation", "ecoinvent name", "mask name", "mask reference product"]
    )
pe2se

,abbreviation,ecoinvent name,mask name,mask reference product
0,pegas.seel.ngcc,"electricity production, natural gas, combined ...",NaN,NaN
1,pegas.seel.ngccc,"electricity production, at power plant/natural...",NaN,NaN
2,pegas.seel.ngt,"electricity production, natural gas, conventio...",NaN,NaN
3,pegas.segafos.gastr,NaN,NaN,NaN
4,pegas.seel.gaschp,"heat and power co-generation, natural gas, con...",NaN,NaN
5,pegas.sehe.gashp,"heat production, natural gas, at industrial fu...",NaN,NaN
6,pegas.seh2.gash2,"hydrogen production, auto-thermal reforming of...",NaN,NaN
7,pegas.seh2.gash2c,"hydrogen production, auto-thermal reforming of...",NaN,NaN
8,pegas.seliqfos.gasftrec,NaN,NaN,NaN
9,pegas.seliqfos.gasftcrec,NaN,NaN,NaN


In [3]:
pe_extraction = pd.read_excel(
    "../mappings/mapping.xlsx", sheet_name="pe extraction", nrows=12, 
    usecols=["entyPe", "ecoinvent name", "mask name", "mask reference product"]
    )
pe_extraction

,entyPe,ecoinvent name,mask name,mask reference product
0,peoil,"crude oil, at production; crude oil, at produc...",NaN,NaN
1,pegas,natural gas production,NaN,NaN
2,pecoal,hard coal mine operation and hard coal prepara...,NaN,NaN
3,peur,"uranium mine operation, open cast; uranium min...",NaN,NaN
4,pegeo,"deep well drilling, for deep geothermal power;...",NaN,NaN
5,pehyd,hydropower plant construction,NaN,NaN
6,pewin,wind turbine construction,NaN,NaN
7,pesol,concentrated solar power plant construction,NaN,NaN
8,pebiolc,hardwood forestry; softwood forestry,NaN,NaN
9,pebios,NaN,NaN,NaN


## 2. Subset of mapping

Take a subset of pe2se technologies.

In [4]:
some_pe2se = ["ngcc", "pc", "bioigcc", "geohdr", "tnrs", "spv"]
pe2se["tech"] = pe2se["abbreviation"].str.split(".").apply(lambda a: a[-1])
pe2se_subset = pe2se[pe2se["tech"].isin(some_pe2se)]
pe2se_subset

,abbreviation,ecoinvent name,mask name,mask reference product,tech
0,pegas.seel.ngcc,"electricity production, natural gas, combined ...",NaN,NaN,ngcc
14,pecoal.seel.pc,"Hard coal, burned in power plant/PC, no CCS; L...",NaN,NaN,pc
27,pebiolc.seel.bioigcc,"electricity production, at BIGCC power plant, ...",NaN,NaN,bioigcc
38,pegeo.seel.geohdr,"electricity production, deep geothermal",NaN,NaN,geohdr
43,pesol.seel.spv,"electricity production, photovoltaic, commercial",NaN,NaN,spv
46,peur.seel.tnrs,"electricity production, nuclear, pressure wate...",NaN,NaN,tnrs


Filter out unmapped PE extraction activities:

In [5]:
pe_extraction.dropna(axis=0, how="any", subset="ecoinvent name", inplace=True)
pe_extraction

,entyPe,ecoinvent name,mask name,mask reference product
0,peoil,"crude oil, at production; crude oil, at produc...",NaN,NaN
1,pegas,natural gas production,NaN,NaN
2,pecoal,hard coal mine operation and hard coal prepara...,NaN,NaN
3,peur,"uranium mine operation, open cast; uranium min...",NaN,NaN
4,pegeo,"deep well drilling, for deep geothermal power;...",NaN,NaN
5,pehyd,hydropower plant construction,NaN,NaN
6,pewin,wind turbine construction,NaN,NaN
7,pesol,concentrated solar power plant construction,NaN,NaN
8,pebiolc,hardwood forestry; softwood forestry,NaN,NaN


Create yaml files from mappings. At this stage, this is not strictly necessary, but as premise uses such yaml files and we want to use its features, we'll include this step here already.

In [6]:
def yaml_dict_from_dataframe(df, var_name="REMIND name"):
    ydict = {}

    for idx, row in df.iterrows():
        techname = row[var_name].strip()
        aliases = [a.strip() for a in row["ecoinvent name"].split(";")]

        # ecoinvent filters
        data = {
            "ecoinvent_aliases": {"fltr": aliases}
        }

        # ecoinvent masks
        if isinstance(row["mask name"], str):
            masks = [m.strip() for m in row["mask name"].split(";")]
            data["ecoinvent_aliases"].update(
                {"mask": {"name": masks}}
            )
        if isinstance(row["mask reference product"], str):
            masks = [m.strip() for m in row["mask reference product"].split(";")]
            data["ecoinvent_aliases"].update(
                {"mask": {"reference product": masks}}
            )

        ydict[techname] = data

    return ydict

In [7]:
import yaml

pe2se_dict = yaml_dict_from_dataframe(pe2se_subset, var_name="tech")
pe_extraction_dict = yaml_dict_from_dataframe(pe_extraction, var_name="entyPe")

yaml.dump(pe2se_dict, open("first_implementation_pe2se.yml", "w"))
yaml.dump(pe_extraction_dict, open("first_implementation_pe_extraction.yml", "w"))

## 3. Regionalize

The first step is to create a list of potential ecoinvent activities.

In [8]:
from premise.activity_maps import InventorySet, get_mapping
import brightway2 as bw

In [9]:
bw.projects.set_current("test")
eidb = bw.Database("ecoinvent_remind_default_2030")

In [10]:
mapping = InventorySet(eidb)

In [11]:
pe2se_filters = get_mapping("first_implementation_pe2se.yml", "ecoinvent_aliases")
pe_extraction_filters = get_mapping("first_implementation_pe_extraction.yml", "ecoinvent_aliases")

Note: The following part seems not perfectly streamlined. In premise there is `generate_sets_from_filters`, which only gives a dictionary with activity names as values, but for the proxies we need the reference product, too.

So probably we'll customize a bit here.

In [12]:
pe2se_activities = {
    tech: mapping.act_fltr(eidb, **fltr) for tech, fltr in pe2se_filters.items()
}
pe_extraction_activities = {
    tech: mapping.act_fltr(eidb, **fltr) for tech, fltr in pe_extraction_filters.items()
}

Now we have to find a proxy for each REMIND region.

In [13]:
import wurst
from wurst import searching as ws
from premise.geomap import Geomap

remind_regions = [
    "CAZ",
    "CHA",
    "EUR",
    "IND",
    "JPN",
    "LAM",
    "MEA",
    "NEU",
    "OAS",
    "REF",
    "SSA",
    "USA"
]

geo = Geomap(model="remind")
ecoinvent_to_iam_loc = {
            loc: geo.ecoinvent_to_iam_location(loc)
            for loc in list(set([a["location"] for a in eidb]))
        }

Code from `premise.transformation.py`, specifically the class `BaseTransformation`, but without changing the database (maybe later if we decide to create representative activities, we can fill in these parts).

In [14]:
def get_region_to_proxy_dataset_mapping(db, name, ref_prod, regions=None):
    d_map = {
            ecoinvent_to_iam_loc[d["location"]]: d["location"]
            for d in ws.get_many(
                db,
                ws.equals("name", name),
                ws.contains("reference product", ref_prod),
            )
            if d["location"] not in remind_regions
        }

    if not regions:
        regions = remind_regions

    if "RoW" in d_map.values():
        fallback_loc = "RoW"
    else:
        if "GLO" in d_map.values():
            fallback_loc = "GLO"
        else:
            fallback_loc = list(d_map.values())[0]

    return {region: d_map.get(region, fallback_loc) for region in regions}

def dataset_proxy(db, name, ref_prod, regions=None):
    d_iam_to_eco = get_region_to_proxy_dataset_mapping(
            db=db, name=name, ref_prod=ref_prod, regions=regions
        )

    d_act = {}

    ds_name, ds_ref_prod = [None, None]

    for region in d_iam_to_eco:

        try:
            dataset = ws.get_one(
                db,
                ws.equals("name", name),
                ws.contains("reference product", ref_prod),
                ws.equals("location", d_iam_to_eco[region]),
            )
        except ws.MultipleResults as err:
            print(
                err,
                "A single dataset was expected, "
                f"but found more than one for: {name, ref_prod}",
            )

        d_act[region] = dataset

    return d_act

Testing the adapted functions.

In [15]:
testname = pe2se_activities["ngcc"][0]["name"]
testprod = pe2se_activities["ngcc"][0]["reference product"]

dataset_proxy(eidb, testname, testprod)

{'CAZ': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, AU, None),
 'CHA': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, CN-JL, None),
 'EUR': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None),
 'IND': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, IN-GA, None),
 'JPN': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, JP, None),
 'LAM': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, BR-Mid-western grid, None),
 'MEA': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, IR, None),
 'NEU': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, NO, None),
 'OAS': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, KR, None),
 'REF': 'electricity production, natural gas, combined cycle power p

Now apply them to the whole lot of technologies. For now, we just take the first activity that we got for each technology. So for example, we just take `Hard coal, burned in power plant/PC, no CCS` but not `Lignite, burned in power plant/PC, no CCS`. This is in principle a method of aggregating several ecoinvent activities into a mix; later, we'll use data to create these mixes.

In [16]:
all_activities = {}

for tech, actlst in pe2se_activities.items():
    name = actlst[0]["name"]
    ref_prod = actlst[0]["reference product"]

    all_activities[tech] = dataset_proxy(eidb, name, ref_prod)

for tech, actlst in pe_extraction_activities.items():
    name = actlst[0]["name"]
    ref_prod = actlst[0]["reference product"]

    all_activities[tech] = dataset_proxy(eidb, name, ref_prod)

# 4. LCIA

Now we do the impact calculation. 

We'll apply the EF method, and subtract activities that we also assess using "approach C", so creating a technosphere where the respective products do not exist. (Note: It makes sense to later create a test for this subtraction. So testing that impacts are not negative, and maybe more.)

In [41]:
method_list = [
    "EF v3.0, acidification, accumulated exceedance (ae)",
    "EF v3.0, climate change, global warming potential (GWP100)",
    "EF v3.0, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe)",
    "EF v3.0, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels",
    "EF v3.0, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P)",
    "EF v3.0, eutrophication: marine, fraction of nutrients reaching marine end compartment (N)",
    "EF v3.0, eutrophication: terrestrial, accumulated exceedance (AE)",
    "EF v3.0, human toxicity: carcinogenic, comparative toxic unit for human (CTUh)",
    "EF v3.0, human toxicity: non-carcinogenic, comparative toxic unit for human (CTUh)",
    "EF v3.0, ionising radiation: human health, human exposure efficiency relative to u235",
    "EF v3.0, land use, soil quality index",
    "EF v3.0, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves)",
    "EF v3.0, ozone depletion, ozone depletion potential (ODP)",
    "EF v3.0, particulate matter formation, impact on human health",
    "EF v3.0, photochemical ozone formation: human health, tropospheric ozone concentration increase",
    "EF v3.0, water use, user deprivation potential (deprivation-weighted water consumption)"
]

midpoints = [m.split(",")[1].strip() for m in method_list]
midpoints

['acidification',
 'climate change',
 'ecotoxicity: freshwater',
 'energy resources: non-renewable',
 'eutrophication: freshwater',
 'eutrophication: marine',
 'eutrophication: terrestrial',
 'human toxicity: carcinogenic',
 'human toxicity: non-carcinogenic',
 'ionising radiation: human health',
 'land use',
 'material resources: metals/minerals',
 'ozone depletion',
 'particulate matter formation',
 'photochemical ozone formation: human health',
 'water use']

In [45]:
methods = []
for mp in midpoints:
    for m in bw.methods:
        if m[0] == "EF v3.0" and mp == m[1]:
            methods.append(m)
methods

[('EF v3.0', 'acidification', 'accumulated exceedance (ae)'),
 ('EF v3.0', 'climate change', 'global warming potential (GWP100)'),
 ('EF v3.0',
  'ecotoxicity: freshwater',
  'comparative toxic unit for ecosystems (CTUe) '),
 ('EF v3.0',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('EF v3.0',
  'eutrophication: freshwater',
  'fraction of nutrients reaching freshwater end compartment (P)'),
 ('EF v3.0',
  'eutrophication: marine',
  'fraction of nutrients reaching marine end compartment (N)'),
 ('EF v3.0', 'eutrophication: terrestrial', 'accumulated exceedance (AE) '),
 ('EF v3.0',
  'human toxicity: carcinogenic',
  'comparative toxic unit for human (CTUh) '),
 ('EF v3.0',
  'human toxicity: non-carcinogenic',
  'comparative toxic unit for human (CTUh) '),
 ('EF v3.0',
  'ionising radiation: human health',
  'human exposure efficiency relative to u235'),
 ('EF v3.0', 'land use', 'soil quality index'),
 ('EF v3.0',
  'material resources: 

In [72]:
import numpy as np

impacts = []
techs = list(all_activities.keys())

counter = 0
for tech, actdict in all_activities.items():
    # create list of subtracted activities
    to_subtract = []
    for tech2, ddict in all_activities.items():
        if tech2 is not tech:
            to_subtract += list(ddict.values())
    idx = []
    for act in to_subtract:
        idx.append(lca.product_dict[act.key])
    idx = list(set(idx))

    temp1 = []
    counter1 = 0
    for reg, act in actdict.items():
        demand = {act: 1}
        lca = bw.LCA(demand=demand)
        lca.lci()

        # remove other products from technosphere    
        # lca.technosphere_matrix[idx,:] = 0

        # redo lci
        lca.lci_calculation()

        # calculate impacts
        temp2 = []
        counter2 = 0
        for m, mp in zip(methods, midpoints):
            lca.switch_method(m)
            lca.lcia()
            temp2.append(lca.score)
            counter2 += 1
            print("\t\t Method {} of {}. Score: {}".format(counter2, len(methods), lca.score))

        temp1.append(temp2)
        counter1 += 1
        print("\t Region {} of {}".format(counter1, len(remind_regions)))
    
    impacts.append(temp1)
    
    # print progress
    counter += 1
    print("{} of {} technologies done.".format(counter, len(techs)))

		 Method 1 of 16. Score: 0.0015932511878635452
		 Method 2 of 16. Score: 0.17541298998600574
		 Method 3 of 16. Score: 4.519942465516413
		 Method 4 of 16. Score: 2.357060543091843
		 Method 5 of 16. Score: 2.244786369110751e-05
		 Method 6 of 16. Score: 0.0007018119489944756
		 Method 7 of 16. Score: 0.007358675653118896
		 Method 8 of 16. Score: 1.5233091782120256e-10
		 Method 9 of 16. Score: 3.608923145113921e-09
		 Method 10 of 16. Score: 0.015871484510634554
		 Method 11 of 16. Score: 3.047522968713058
		 Method 12 of 16. Score: 4.561582430327629e-06
		 Method 13 of 16. Score: 3.178599670685909e-08
		 Method 14 of 16. Score: 2.0660751047386678e-08
		 Method 15 of 16. Score: 0.0018740343624166324
		 Method 16 of 16. Score: 0.05576137595648637
	 Region 1 of 12
		 Method 1 of 16. Score: 0.0015932511878635452
		 Method 2 of 16. Score: 0.17541298998600574
		 Method 3 of 16. Score: 4.519942465516413
		 Method 4 of 16. Score: 2.357060543091843
		 Method 5 of 16. Score: 2.24478636911075

KeyboardInterrupt: 

In [51]:
import xarray as xr

impacts_np = np.array(impacts)

impacts_xr = xr.DataArray(
    impacts_np,
    dims=["technology", "region", "midpoint"],
    coords = {
        "technology": techs,
        "region": remind_regions,
        "midpoint": midpoints
    }
)

impacts_xr

<xarray.DataArray (technology: 15, region: 12, midpoint: 16)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * technology  (technology) <U7 'bioigcc' 'geohdr' 'ngcc' ... 'peur' 'pewin'
  * region      (region) <U3 'CAZ' 'CHA' 'EUR' 'IND' ... 'OAS' 'REF' 'SSA' 'USA'
  * midpoint    (midpoint) <U43 'acidification' 'climate change' ... 'water use'

In [53]:
np.sum(np.isnan(impacts_np))

2880

In [64]:
lca = bw.LCA(demand=demand)
lca.lci()
A = lca.technosphere_matrix

In [65]:
A[[0, 4, 5], :] = np.nan

c:\Users\davidba\.conda\envs\bw2\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [69]:
Acopy = A.tolil()

In [70]:
Acopy[[4,5,7,3,8], :] = 0

In [71]:
A[[4,5,7,3,8], :] = 0

c:\Users\davidba\.conda\envs\bw2\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
